In [11]:
import numpy as np

from numpy.testing import assert_allclose


# Part I. Construct a Householder reflection of a vector.


Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$


Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.


In [12]:
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector

    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)

    # ... ENTER YOUR CODE HERE ...
    y = np.zeros_like(vec)
    y[0] = np.linalg.norm(vec)
    u = (vec - y)/np.linalg.norm(vec - y)
    H = np.eye(len(vec))-(2*np.outer(u, u))
    outvec = np.dot(H, vec)
    return outvec, H


Test your function using tests below:


In [13]:
# Test I.1 (10% of the total grade).

v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(np.dot(h, v1), v)
assert_allclose(np.dot(h, v), v1)


In [14]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)


# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$.


Write a function, which receives a recangular matrix, $A$, and returns the Q and R factors of the $QR$ factorization of $A$.


In [15]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.

    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix

    Returns
    -------
    q: ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix

    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)

    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape

    # ... ENTER YOUR CODE HERE ...
    a1 = a
    m, n = a1.shape
    q = np.eye(m)
    for i in range(n):
        _, H = householder(a1[i:, i])
        H1 = np.eye(m)
        H1[i:, i:] = H
        a1 = np.dot(H1, a1)
        q = np.dot(q, H1)
    return q, a1


In [16]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)


In [17]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)


Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)


In [18]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)


Check if your `q` and `r` agree with `qq` and `rr`. Explain.


_Enter your explanation here_ (10% of the total grade, peer-graded)


# Part III. Avoid forming Householder matrices explicitly.


Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$

while

$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to _avoid_ forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).


Write a function which constructs the `QR` decomposition of a matrix _without ever forming the_ $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors.

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests.

(Peer-graded, 40% of the total grade)


$$
\mathbf{R} = \mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A}
$$

$$
\mathbf{H}_i = \mathbf{I} - 2 \mathbf{u}_i \mathbf{u}_i^T
$$

$$
\mathbf{R}_0 =  \mathbf{A}
$$

$$
\mathbf{R}_1 = \mathbf{H}_1 \mathbf{R}_0= \ (\mathbf{I} - 2 \mathbf{u}_1 \mathbf{u}_1^T) \mathbf{R}_0 =  \mathbf{R}_0 -  2 \mathbf{u}_1 ( \mathbf{u}_1^T  \mathbf{R}_0)
$$

$$
\mathbf{R}_2 = \mathbf{H}_2 \mathbf{R}_1 = \ (\mathbf{I} - 2 \mathbf{u}_2 \mathbf{u}_2^T) \mathbf{R}_1 =  \mathbf{R}_1 -  2 \mathbf{u}_2 ( \mathbf{u}_2^T  \mathbf{R}_1)
$$

$$
\mathbf{R} = \mathbf{H}_n \mathbf{R}_{n-1} =  (\mathbf{I} - 2 \mathbf{u}_{n-1} \mathbf{u}_{n-1}^T) \mathbf{R}_{n-1} = \mathbf{R}_{n-1} -  2 \mathbf{u}_{n-1} ( \mathbf{u}_{n-1}^T  \mathbf{R}_{n-1})
$$


In [19]:
def qr_decomp_f(a):
    """Compute R without ever forming the  H  matrices

    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix

    Returns
    -------

    R : ndarray, shape(m, n)
        The upper triangular matrix
    vecs:  reflection vectors.

    """
    # ... ENTER YOUR CODE HERE ...
    m, n = a.shape
    R = a
    vecs = np.array([])
    for i in range(0, n):
        norm = np.linalg.norm(a[i:, i])
        y = np.zeros(m - i)
        y[0] = norm
        #vec, _ = householder(a[i:, i])
        x = a[i:, i]
        u = (x - y) / np.linalg.norm(x - y)
        R[i:, i:] = R[i:, i:] - 2 * \
            u.reshape(-1, 1) @ (u.reshape(1, -1) @ R[i:, i:])
        vecs = np.append(vecs, u)
    return R, vecs


In [21]:
rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
r, vecs = qr_decomp_f(a)

q, qt = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a, atol=1e-10)


In [23]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)
